In [6]:
import numpy as np
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

model = keras.models.load_model(r"C:\Users\yuvat\project SLD\best_model_data1.h5")
background = None
accumulated_weight = 0.5

word_dict = {0:'Zero',1:'One',2:'Two',3:'Three',4:'Four',5:'Five',6:'Six',7:'Seven',8:'Eight',9:'Nine',10:'A',11:'B',12:'C',13:'D',14:'E',15:'F',16:'G',17:'H',18:'I',19:'J',20:'K',21:'L',22:'M',23:'N',24:'O',25:'P',26:'Q',27:'R',28:'S',29:'T',30:'U',31:'V',32:'W',33:'X',34:'Y',35 :'Z'}

ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

In [7]:
def cal_accum_avg(frame, accumulated_weight):
    
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [8]:
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    
    #_ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    thresholded = cv2.adaptiveThreshold(diff,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    ret, thresholded = cv2.threshold(thresholded, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    #grab  contours from frame
    contours, hierachy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #if length of contours list=0, means we didn't get any contours... 
    if len(contours) == 0:
        return None
    else:
        # the largest external contors should be the hand
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        #returning the hand segment and thresholded image
        return (thresholded, hand_segment_max_cont)

In [9]:
cam = cv2.VideoCapture(0)
num_frames = 0
while True:
    ret, frame = cam.read()
    
    #flipping the frame to prevent inverted image
    frame = cv2.flip(frame, 1)
    
    frame_copy = frame.copy()
    
    #ROI from the frame
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
    
    gray_frame = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray_frame, (5,5),0)

    
    if num_frames < 70:
        cal_accum_avg(blur, accumulated_weight)
        cv2.putText(frame_copy, "FETCHING BACKGROUND.. PLEASE WAIT", (80,400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        
    else:
        #segmenting the hand region
        hand = segment_hand(blur)
        
        
        #checking if we are able to detect the hand...
        if hand is not None:
            thresholded, hand_segment = hand
            
            #drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right,ROI_top)], -1, (255, 0, 0),1)
            cv2.imshow("Thresholded Hand Image", thresholded)
            thresholded = cv2.resize(thresholded, (64, 64))
            thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2RGB)
            thresholded = np.reshape(thresholded, (1, thresholded.shape[0], thresholded.shape[1], 3))
            
            pred = model.predict(thresholded)
            cv2.putText(frame_copy, word_dict[np.argmax(pred)], (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
    #draw ROI on frame_copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,ROI_bottom), (255,128,0), 3)
            
    #incrementing the numner of frames for tracking
    num_frames += 1
            
    #display the frame with segmanted hand
    cv2.putText(frame_copy, "HAND SIGN RECOGNITION_ _ _ _", (10,20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign Detection", frame_copy)
            
            
    #close windows with ESC
    k = cv2.waitKey(1) & 0xFF
            
    if k == 27:
        break
                
#release the camera and destroy all windows
cam.release()
cv2.destroyAllWindows()
        
    